# CS 682 Project Notebook
### Project Title: Friendly Streets: A Street Classifier for Cautious Cyclists
### Authors: Josh Sennett, Evan Rourke
### Fall 2018 - Professor Erik Learned-Miller

This notebook is to be used as a central location to write code for the project.


## Imports and Setup

In [1]:
# Intialize the scenario
# Import libraries
import torch
import torch.nn as nn
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import os
import numpy as np
from scipy.misc import imresize as imresize
from imageio import imread
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import copy
import random as rdm
from PIL import Image


import sys 
sys.path.append("../../")

## Load Image Labels for Places 365

In [2]:
def load_labels():
    # prepare all the labels
    # scene category relevant
    file_name_category = 'categories_places365.txt'
    # if not os.access(file_name_category, os.W_OK):
    #     synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
    #     os.system('wget ' + synset_url)
    classes = list()
    with open(file_name_category) as class_file:
        for line in class_file:
            classes.append(line.strip().split(' ')[0][3:])
    classes = tuple(classes)

    # indoor and outdoor relevant
    file_name_IO = 'IO_places365.txt'
    # if not os.access(file_name_IO, os.W_OK):
    #     synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/IO_places365.txt'
    #     os.system('wget ' + synset_url)
    with open(file_name_IO) as f:
        lines = f.readlines()
        labels_IO = []
        for line in lines:
            items = line.rstrip().split()
            labels_IO.append(int(items[-1]) -1) # 0 is indoor, 1 is outdoor
    labels_IO = np.array(labels_IO)

    # scene attribute relevant
    file_name_attribute = 'labels_sunattribute.txt'
    # if not os.access(file_name_attribute, os.W_OK):
    #     synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/labels_sunattribute.txt'
    #     os.system('wget ' + synset_url)
    with open(file_name_attribute) as f:
        lines = f.readlines()
        labels_attribute = [item.rstrip() for item in lines]
    file_name_W = 'W_sceneattribute_wideresnet18.npy'
    # if not os.access(file_name_W, os.W_OK):
    #     synset_url = 'http://places2.csail.mit.edu/models_places365/W_sceneattribute_wideresnet18.npy'
    #     os.system('wget ' + synset_url)
    W_attribute = np.load(file_name_W)

    return classes, labels_IO, labels_attribute, W_attribute


In [3]:
def hook_feature(module, input, output):
    features_blobs.append(np.squeeze(output.data.cpu().numpy()))

# Transform to CAM

This function returns the Class Activation Map (CAM)

In [4]:
def returnCAM(feature_conv, weight_softmax, class_idx):
    # generate the class activation maps upsample to 256x256
    size_upsample = (256, 256)
    nc, h, w = feature_conv.shape
    output_cam = []
    for idx in class_idx:
        cam = weight_softmax[class_idx].dot(feature_conv.reshape((nc, h*w)))
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(imresize(cam_img, size_upsample))
    return output_cam

In [5]:
def returnTF():
# load the image transformer
    tf = trn.Compose([
        trn.Resize((224,224)),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    return tf

## Load the model

In [6]:
def load_model():
    # this model has a last conv feature map as 14x14

    model_file = 'wideresnet18_places365.pth.tar'
    # if not os.access(model_file, os.W_OK):
    #     os.system('wget http://places2.csail.mit.edu/models_places365/' + model_file)
    #     os.system('wget https://raw.githubusercontent.com/csailvision/places365/master/wideresnet.py')

    import wideresnet
    model = wideresnet.resnet18(num_classes=365)
    checkpoint = torch.load(model_file, map_location=lambda storage, loc: storage)
    state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
    model.load_state_dict(state_dict)
    model.eval()



    # the following is deprecated, everything is migrated to python36

    ## if you encounter the UnicodeDecodeError when use python3 to load the model, add the following line will fix it. Thanks to @soravux
    #from functools import partial
    #import pickle
    #pickle.load = partial(pickle.load, encoding="latin1")
    #pickle.Unpickler = partial(pickle.Unpickler, encoding="latin1")
    #model = torch.load(model_file, map_location=lambda storage, loc: storage, pickle_module=pickle)

    model.eval()
    # hook the feature extractor
    features_names = ['layer4','avgpool'] # this is the last conv layer of the resnet
    for name in features_names:
        model._modules.get(name).register_forward_hook(hook_feature)
    return model


## Initialize a bunch of stuff and define a random image loader

In [7]:
c = {'0': 'Not Bike Friendly',
     '1': 'Bike Friendly'
    }

datadir = '../data/images/'
camdir = '../cam/'
testdir = datadir + 'test/'
traindir = datadir + 'train/'

def load_random_image():
    # load a random classifier, either bike friendly or not bike friendly
    y = rdm.choice(list(c.keys()))
    
    items = os.listdir(traindir + y)
    
    imgname = rdm.choice(items)
    imgdir = traindir + y + '/' + imgname
    
    img = Image.open(imgdir)
#     print(imgdir)
#     print(img)
    return img, y, imgname

# image, _, _ = load_random_image()
# plt.figure(figsize=(6,6))
# plt.imshow(image)
# plt.axis('off')
# plt.show()

## The following is used to generate a bunch of Class Activation Maps

In [11]:
# input_images 
images = []
img_classes = []
img_names = []


# Perform CAM on N number of images
N = 10


# Load 10 random images
for i in range(N):
    loaded_image, y, img_name = load_random_image()
    img_name = img_name[:-4]
    images.append(loaded_image)
    img_names.append(img_name)
    img_classes.append(y)
        
for j in range(len(images)):
    # load the labels
    classes, labels_IO, labels_attribute, W_attribute = load_labels()

    # load the model
    features_blobs = []
    model = load_model()

    # load the transformer
    tf = returnTF() # image transformer

    # get the softmax weight
    params = list(model.parameters())
    weight_softmax = params[-2].data.numpy()
    weight_softmax[weight_softmax<0] = 0
    
    
    
#     print(img_names[j])
    img = images[j]
    input_img = V(tf(img).unsqueeze(0))
    
    # forward pass
    logit = model.forward(input_img)
    h_x = F.softmax(logit, 1).data.squeeze()
    probs, idx = h_x.sort(0, True)
    probs = probs.numpy()
    idx = idx.numpy()
    
    # save the indoor/outdoor prediction
    io_image = np.mean(labels_IO[idx[:10]]) # vote for the indoor or outdoor
    indoorOrOutdoor = []
    if io_image < 0.5:
        indoorOrOutdoor.append('--TYPE OF ENVIRONMENT: indoor')
    else:
        indoorOrOutdoor.append('--TYPE OF ENVIRONMENT: outdoor')
        

    # save the prediction of scene category
    sceneCategories = []
    sceneCategories.append('--SCENE CATEGORIES:')
    for k in range(0, 5):
        sceneCategories.append('{:.3f} -> {}'.format(probs[k], classes[idx[k]]))

    # save the scene attributes
    responses_attribute = W_attribute.dot(features_blobs[1])
    idx_a = np.argsort(responses_attribute)
    sceneAttributes = []
    sceneAttributes.append('--SCENE ATTRIBUTES:')
    for k in range(-1,-10,-1):
        sceneAttributes.append('{}'.format(labels_attribute[idx_a[k]]))


    # generate class activation mapping
    CAMs = returnCAM(features_blobs[0], weight_softmax, [idx[0]])

    
    # Set the directory of where to read the images
    imgdir = '../data/images/train/' + img_classes[j] + '/' + img_names[j] + '.jpg'
    
    # read the image with cv2
    img = cv2.imread(imgdir)
    height, width, _ = img.shape
    heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_JET)

    heatmap_brightness = 0.4
    image_brightness = 0.5
    
    # Generate heat map
    result = (heatmap * heatmap_brightness) + (img * image_brightness)
    
    # Write the heatmap and original image to the cam directory
    print("Writing to", camdir + img_classes[j] + '/' + img_names[j] + '_cam.jpg')
    cv2.imwrite(camdir + img_classes[j] + '/' + img_names[j] + '_cam.jpg', result)
    cv2.imwrite(camdir + img_classes[j] + '/' + img_names[j] + '.jpg', img)

    # Write all of the attributes to the cam directory as well
    with open(camdir + img_classes[j] + '/' + img_names[j] + '.txt', 'w') as f:
        for item1 in indoorOrOutdoor:
            f.write("%s\n" %item1)
        f.write("\n")
        for item2 in sceneCategories:
            f.write("%s\n" %item2)
        f.write("\n")
        for item3 in sceneAttributes:
            f.write("%s\n" %item3)


/home/evan_rourke_er/.conda/envs/cs682/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if sys.path[0] == '':


Writing to ../cam/0/w392696868_n4829697984_cam.jpg
Writing to ../cam/1/w193816728_n2043277804_cam.jpg
Writing to ../cam/1/w261756364_n5172284377_cam.jpg
Writing to ../cam/1/w126301280_n40465534_cam.jpg
Writing to ../cam/0/w428017300_n3672285804_cam.jpg
Writing to ../cam/0/w165327264_n4218467368_cam.jpg
Writing to ../cam/0/w533092481_n244284888_cam.jpg
Writing to ../cam/0/w5534532_n1395454704_cam.jpg
Writing to ../cam/1/w335460685_n133096015_cam.jpg
Writing to ../cam/1/w28714075_n5589623230_cam.jpg
